In [1]:
import os
from langchain_community.document_loaders import UnstructuredURLLoader, MergedDataLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader, DirectoryLoader
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
import logging

from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from src.prompt_template import *

In [2]:
def load_data_source(loaded_files):
    for loaded_file in loaded_files:
        print('loaded_file - ', loaded_file)
        # temp_file = create_temp_file(loaded_file)
        temp_file = './tmp/yolo.pdf'
        loader = PyPDFLoader(temp_file)
        print('loader - ', loader)
        data = loader.load()
        print('data - ', data)
        return data



In [3]:
def get_data_chunks(data):
    recursive_char_text_splitter=RecursiveCharacterTextSplitter(
                                                chunk_size=800,
                                                chunk_overlap=100)
        # text_chunks=recursive_char_text_splitter.split_documents(documents)
    documents=recursive_char_text_splitter.split_documents(data)
    print('documents - ', documents)
    print('documents type - ', type(documents))
    print('documents length - ', len(documents))
    return documents


In [4]:
def create_embeddings():
    embeddings=HuggingFaceEmbeddings(
            model_name='sentence-transformers/all-MiniLM-L6-v2', 
            model_kwargs={'device':'cpu'}
    )
    return embeddings

In [5]:
def store_data_in_vectordb(documents, embeddings):
    knowledge_base =FAISS.from_documents(documents, embeddings)
    print('knowledge_base - ', knowledge_base)
    return knowledge_base

In [6]:
def get_llm_model():
     
    llm=CTransformers(
            model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
            model_type="llama",
            config={'max_new_tokens':200,
                    'temperature':0.01}
    )
    return llm


In [7]:
def get_prompt():
    template="""Use the following pieces of information to answer the user's question.
            If you dont know the answer just say you know, don't try to make up an answer.

            Context:{context}
            Question:{question}

            Only return the helpful answer below and nothing else
            Helpful answer
            """
        

    prompt = PromptTemplate(template=template, input_variables=["context", "question"])
    return prompt

In [8]:
def create_chain(llm, vector_store, prompt):
    chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type='stuff',
            retriever=vector_store.as_retriever(search_kwargs={'k': 2}),
            return_source_documents=False,
            chain_type_kwargs={'prompt': prompt}
    )
    return chain

In [9]:
def run_query(chain):
    result=chain({'query':'Tell me about LORA'})
    print(f"Answer:{result['result']}")

In [10]:
data = load_data_source(['./data/yolo.pdf'])
documents = get_data_chunks(data)
embeddings = create_embeddings()
vector_store_data = store_data_in_vectordb(documents, embeddings)
llm_model = get_llm_model()
prompt = get_prompt()
chain = create_chain(llm_model, vector_store_data, prompt)


loaded_file -  ./data/yolo.pdf


NameError: name 'create_temp_file' is not defined

In [ ]:
run_query(chain)

Number of tokens (1437) exceeded maximum context length (512).


Number of tokens (1438) exceeded maximum context length (512).
Number of tokens (1439) exceeded maximum context length (512).
Number of tokens (1440) exceeded maximum context length (512).
Number of tokens (1441) exceeded maximum context length (512).
Number of tokens (1442) exceeded maximum context length (512).
Number of tokens (1443) exceeded maximum context length (512).
Number of tokens (1444) exceeded maximum context length (512).
Number of tokens (1445) exceeded maximum context length (512).
Number of tokens (1446) exceeded maximum context length (512).
Number of tokens (1447) exceeded maximum context length (512).
Number of tokens (1448) exceeded maximum context length (512).
Number of tokens (1449) exceeded maximum context length (512).
Number of tokens (1450) exceeded maximum context length (512).
Number of tokens (1451) exceeded maximum context length (512).
Number of tokens (1452) exceeded maximum context length (512).
Number of tokens (1453) exceeded maximum context length

Answer:
The user:help
The user:help
The user:help
The user:help
The user:help
The user:help
The user: You can you are there are you have any suggestions for each model:help
The user:help
The user:help
The user:help
The user:help
The user:help
The user:help
The user:help
The user:help
The user:help
The user:help
The user:help
The user:help
The user:help
The user:help
The user:help
The user:help
The user:help
The user:help
The user:help
The user:help
The user:help
The user:help
The user:help
The user:help
The user:help
The user:help
The user:help
The user:help
The user:help
The user:help

